## Continuous Pre-training using LLM recipie

In [ ]:
! pip install datasets transformers torch tensorboard

In [1]:
# ====================================================IMPORT=================================================================

from datasets import DatasetDict, load_dataset, load_from_disk, Dataset

import os
import warnings
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
)
import time
from torch.utils.tensorboard import SummaryWriter

warnings.filterwarnings("ignore")
os.environ.update(
    {
        # "NCCL_P2P_DISABLE": "1",
        # "NCCL_IB_DISABLE": "1",
        # "TOKENIZERS_PARALLELISM": "false",
        # "CUDA_VISIBLE_DEVICES": "3,2,1",
    }
)

from datasets import load_from_disk
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
import torch
import pandas as pd
import sys
import os
from datasets import Dataset
import argparse


# ========================== CMD Argument Parser ==========================
# def parse_args():
#     parser = argparse.ArgumentParser(description="Train a model using CPT (Continual Pretraining Training)")
#     parser.add_argument("--per_device_train_batch_size", type=int, default=10, help="Batch size per device during training")
#     parser.add_argument("--per_device_eval_batch_size", type=int, default=10, help="Batch size per device during evaluation")
#     parser.add_argument("--num_train_epochs", type=int, default=5, help="Number of training epochs")
#     parser.add_argument("--learning_rate", type=float, default=1e-6, help="Learning rate for training")
#     parser.add_argument("--project_root", type=str, default="/Users/lujun.li/projects/mt_luxembourgish", help="Path to project root")
#     parser.add_argument("--training_dataset_path", type=str, default="data/processed/dataset_merged_llama_fake_targets.jsonl", help="Path to training dataset")
#     parser.add_argument("--model_name", type=str, default="/home/llama/Personal_Directories/srb/binary_classfication/Llama-3.2-3B-Instruct", help="Path to model")
#     parser.add_argument("--resume_from_checkpoint", type=bool, default=False, help="Resume training from checkpoint")
#     parser.add_argument("--resume_checkpoint_path", type=str, default=None, help="Path to checkpoint to resume training from")
#     return parser.parse_args()

# args = parse_args()

print("Arguments passed:")
# print(f"Train Batch Size: {args.per_device_train_batch_size}")
# print(f"Eval Batch Size: {args.per_device_eval_batch_size}")
# print(f"Number of Epochs: {args.num_train_epochs}")
# print(f"Learning Rate: {args.learning_rate}")
# print(f"Project Root: {args.project_root}")
# print(f"Training Dataset Path: {args.training_dataset_path}")
# print(f"Model Name: {args.model_name}")

# learning_rate = args.learning_rate # Learning rate for the optimizer
# per_device_train_batch_size = args.per_device_train_batch_size  # Batch size for training per device
# per_device_eval_batch_size = args.per_device_eval_batch_size  # Batch size for evaluation per device
# num_train_epochs = args.num_train_epochs  # Number of epochs for training
# training_dataset_path = args.training_dataset_path
# project_root = args.project_root
# model_name = args.model_name
# resume_from_checkpoint = args.resume_from_checkpoint
# resume_checkpoint_path = args.resume_checkpoint_path

learning_rate = 1e-6 # Learning rate for the optimizer
per_device_train_batch_size = 1  # Batch size for training per device
per_device_eval_batch_size = 1  # Batch size for evaluation per device
num_train_epochs = 1  # Number of epochs for training
training_dataset_path = "data/training_dataset/dataset_GPT_split.jsonl"
project_root = "/home/snt/projects_lujun/mt_luxembourgish"
model_name = "/home/snt/llm_models/Llama-3.2-1B-Instruct"
resume_from_checkpoint = False
resume_checkpoint_path = None
device="cuda:0"

if resume_from_checkpoint and resume_checkpoint_path is None:
    raise ValueError("Please provide a checkpoint path to resume training from")


train_ratio = 0.001  # Number of samples to be used for training and evaluation
learning_rate = 1e-6  # Learning rate for the optimizer
per_device_train_batch_size = 1  # Batch size for training per device
per_device_eval_batch_size = 1  # Batch size for evaluation per device
num_train_epochs = 5  # Number of epochs for training
weight_decay = 0.01  # Weight decay rate for regularization
MAX_LEN = 512  # Maximum sequence length for model inputs
warmup_ratio = 0.5

logging_steps = 100
evaluation_strategy="epoch"
save_strategy="epoch"
eval_steps=500

max_grad_norm = 0.3
fp16 = True
resume_from_checkpoint = False

project_root = "/home/snt/projects_lujun/mt_luxembourgish"
model_name = (
    # "/home/llama/Personal_Directories/srb/binary_classfication/Llama-3.2-3B-Instruct"
    "/home/snt/llm_models/Llama-3.2-1B-Instruct"
)
val_dataset_path = os.path.abspath(os.path.join(project_root, "data/fake_targets/flores_devtest_arrow"))
train_dataset_path = os.path.abspath(os.path.join(project_root, "data/training_dataset/dataset_GPT_split.jsonl"))
sys.path.append(project_root)

/home/snt/miniconda3/envs/mt_lux_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Arguments passed:


In [ ]:
# ========================== Main Training Code ==========================

val_dataset_path = os.path.abspath(os.path.join(project_root, "data/fake_targets/flores_devtest_arrow"))
train_dataset_path = os.path.abspath(os.path.join(project_root, training_dataset_path))
sys.path.append(project_root)

# Load dataset
if train_dataset_path.endswith(".jsonl"):
    dataset = Dataset.from_json(train_dataset_path)  # Ensure correct format
else:
    dataset = load_from_disk(train_dataset_path)

# Filter by split
train_dataset = dataset.filter(lambda x: x["split"] == "train")
val_dataset = dataset.filter(lambda x: x["split"] == "val")


# Select subset
train_dataset = train_dataset.select(range(int(len(train_dataset) * train_ratio)))
val_dataset = val_dataset.select(range(int(len(val_dataset) * train_ratio)))  # Avoid out-of-range error

# Rename columns
train_dataset = train_dataset.rename_columns({
    "input": "Luxembourgish",
    "translated_text": "English",
})

val_dataset = val_dataset.rename_columns({
    "input": "Luxembourgish",
    "translated_text": "English",
})


tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


def create_prompt(
    sample, mode="train", src_lng="English", tgt_lng="Luxembourgish", tokenizer=None
):
    if tokenizer is None or tokenizer.eos_token is None:
        raise ValueError("A tokenizer with a defined EOS token is required.")

    system_message = f"You are a helpful AI assistant for translation."
    input_text = sample[src_lng.capitalize()].strip()  # Extract the input text.
    response = (
        sample[tgt_lng.capitalize()].strip() if tgt_lng.capitalize() in sample else ""
    )  # Extract the target text.

    # Get the EOS token from the tokenizer.
    eos_token = tokenizer.eos_token

    # Construct the full prompt.
    full_prompt = (

        "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n" + system_message + "<|eot_id|>" +  # System Message

        "<|start_header_id|>user<|end_header_id|>\n\n" + f"Translate the English input text into Luxembourgish.\n{input_text}. Do not include any additional information or unrelated content."  +  "<|eot_id|><|start_header_id|>assistant<|end_header_id|>" # User Query
    )

    full_input_prompt = full_prompt
    full_input_CPT_prompt = full_input_prompt + response + "<|eot_id|>"
    labels = response + "<|eot_id|>"  # <|eot_id|> is end of generation
    if mode == "train":
        full_prompt += response + eos_token

    return {"full_input_prompt": full_input_prompt, "full_input_CPT_prompt": full_input_CPT_prompt, "labels": labels}

train_dataset_CPT = train_dataset.map(
    lambda sample: {
        "full_input_CPT_prompt": create_prompt(sample, mode="train", tokenizer=tokenizer)["full_input_CPT_prompt"]
    }
).select_columns(["full_input_CPT_prompt"])

val_dataset_CPT = val_dataset.map(
    lambda sample: {
        "full_input_CPT_prompt": create_prompt(sample, mode="train", tokenizer=tokenizer)["full_input_CPT_prompt"]
    }
).select_columns(["full_input_CPT_prompt"])


# train_dataset_SFT = train_dataset.map(
#     lambda sample: {
#         **create_prompt(sample, mode="train", tokenizer=tokenizer),
#         "inputs": create_prompt(sample, mode="train", tokenizer=tokenizer)["full_input_prompt"],
#         "outputs": create_prompt(sample, mode="train", tokenizer=tokenizer)["labels"]
#     }
# )

# val_dataset_SFT = val_dataset.map(
#     lambda sample: {
#         **create_prompt(sample, mode="train", tokenizer=tokenizer),
#         "inputs": create_prompt(sample, mode="train", tokenizer=tokenizer)["full_input_prompt"],
#         "outputs": create_prompt(sample, mode="train", tokenizer=tokenizer)["labels"]
#     }
# )

# dataset_CPT = DatasetDict({"train": train_dataset_CPT, "val": val_dataset_CPT})
# dataset_SFT = DatasetDict({"train": train_dataset_SFT, "val": val_dataset_SFT})

data_collator = DataCollatorForLanguageModeling(
    tokenizer, mlm=False, return_tensors="pt"
)

def tokenize_function_CPT(examples):
    return tokenizer(
        examples["full_input_CPT_prompt"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN,
        return_tensors="pt",
    )

# def tokenize_function_SFT(examples):
#     tokenized_example = tokenizer(
#         examples["inputs"],
#         truncation=True,
#         padding="max_length",
#         max_length=MAX_LEN,
#         return_tensors="pt",
#     )
#     tokenized_example["labels"] = tokenizer(examples["labels"], truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="pt")["input_ids"]
#     return tokenized_example


tokenized_train_dataset_CPT = train_dataset_CPT.map(
    tokenize_function_CPT, batched=True, remove_columns=["full_input_CPT_prompt"]
)
tokenized_val_dataset_CPT = val_dataset_CPT.map(
    tokenize_function_CPT, batched=True, remove_columns=["full_input_CPT_prompt"]
)

# tokenized_train_dataset_SFT = train_dataset_SFT.map(
#     tokenize_function_SFT, batched=True, remove_columns=["inputs","outputs"]
# )   

# tokenized_val_dataset_SFT = val_dataset_SFT.map(
#     tokenize_function_SFT, batched=True, remove_columns=["inputs","outputs"]
# )


Map: 100%|██████████| 3/3 [00:00<00:00, 283.51 examples/s]


In [3]:
print (train_dataset_CPT[0]["full_input_CPT_prompt"])

<|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for translation.<|eot_id|><|start_header_id|>user<|end_header_id|>

Translate the English input text into Luxembourgish.
In both the women's and men's events, the mass start was on the program as part of the Tour de Ski in Switzerland. The women were challenged on a 10-kilometer circuit, and it was Linn Svahn from Sweden who managed to secure victory in the final sprint. In second place was Russia's Julia Stupak, ahead of the American Jessie Diggins.. Do not include any additional information or unrelated content.<|eot_id|><|start_header_id|>assistant<|end_header_id|>Bei den Dammen a bei den Häre stoung an der Schwäiz de Massestart am Kader vum Tour de Ski um Programm. D'Damme waren op engem Circuit iwwer 10 Kilometer gefuerdert a sou war et d'Linn Svahn aus Schweden, dat sech um Enn am Zilsprint d'Victoire séchere konnt. Op déi zweet Plaz ass d'Julia Stupak aus Russland komm, virun der US-Amerikanerin Jessie Di

In [ ]:
# ====================================================TRAINING=================================================================
def print_trainable_parameters(model):
    """Prints the number of trainable parameters in the model."""
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    return f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"

def train_ddp_accelerate_cpt():
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model.config.use_cache = False
    current = time.time()
    if resume_from_checkpoint:
        output_dir = resume_checkpoint_path
    else:
        input_file_name = training_dataset_path.split("/")[-1].split(".")[0]
        output_dir = f"logs/fit_{current}_{train_ratio}_{input_file_name}"

    print(print_trainable_parameters(model))
    
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        warmup_ratio=warmup_ratio,
        evaluation_strategy=evaluation_strategy,
        save_strategy=save_strategy,
        # eval_steps=eval_steps,
        logging_steps=logging_steps,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        fp16=fp16,
        max_grad_norm=max_grad_norm,
        group_by_length=True,
        lr_scheduler_type="cosine",
        report_to="tensorboard",
        ddp_find_unused_parameters=False,
        remove_unused_columns=False,
        disable_tqdm=False,
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset_CPT,
        eval_dataset=tokenized_val_dataset_CPT,
        tokenizer=tokenizer,
        data_collator=data_collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    trainer.train(resume_from_checkpoint=resume_from_checkpoint)
    print("Finished training CPT.")

# def train_ddp_accelerate_sft():
#     model = AutoModelForCausalLM.from_pretrained(model_name)
#     model.config.use_cache = False
#     current = time.time()
#     if resume_from_checkpoint:
#         output_dir = resume_checkpoint_path
#     else:
#         input_file_name = training_dataset_path.split("/")[-1].split(".")[0]
#         output_dir = f"logs/fit_{current}_{train_ratio}_{input_file_name}"

#     print(print_trainable_parameters(model))
    
#     training_args = TrainingArguments(
#         output_dir=output_dir,
#         num_train_epochs=num_train_epochs,
#         per_device_train_batch_size=per_device_train_batch_size,
#         per_device_eval_batch_size=per_device_eval_batch_size,
#         warmup_ratio=warmup_ratio,
#         evaluation_strategy=evaluation_strategy,
#         save_strategy=save_strategy,
#         # eval_steps=eval_steps,
#         logging_steps=logging_steps,
#         learning_rate=learning_rate,
#         weight_decay=weight_decay,
#         fp16=fp16,
#         max_grad_norm=max_grad_norm,
#         group_by_length=True,
#         lr_scheduler_type="cosine",
#         report_to="tensorboard",
#         ddp_find_unused_parameters=False,
#         remove_unused_columns=False,
#         disable_tqdm=False,
#         load_best_model_at_end=True,
#     )

#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=tokenized_train_dataset_SFT,
#         eval_dataset=tokenized_val_dataset_SFT,
#         tokenizer=tokenizer,
#         data_collator=data_collator,
#         callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
#     )

#     trainer.train(resume_from_checkpoint=resume_from_checkpoint)
#     print("Finished training CPT.")
    

from accelerate import Accelerator


def main():
    accelerator = Accelerator()
    train_ddp_accelerate_cpt()

if __name__ == "__main__":
    main()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


trainable params: 1235814400 || all params: 1235814400 || trainable%: 100.0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss
1,2.807000,2.629273
2,1.916800,2.416902
3,1.282000,2.558568
4,0.775800,2.668485


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Finished training CPT.


In [12]:
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, pipeline
import torch


device = "cuda:0"
model_checkpoint = "/home/snt/projects_lujun/mt_luxembourgish/logs/fit_1740413339.3789797_0.001_dataset_GPT_split/checkpoint-812"
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
system_message = f"You are a helpful AI assistant for translation."
input_text = """The concept of the lead candidate is not without controversy: One pro-argument is that the election becomes more democratic this way. On the other hand, the heads of state and government lose power as a result. Simply put, the EU is responsible for what is agreed upon between it and the member states, for example, for the internal market, trade policy, monetary policy, and competition rules, and there is also a common security and foreign policy.. Do not include any additional information or unrelated content."""
eos_token = tokenizer.eos_token

# Construct the full prompt.
full_prompt = (

        "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n" + system_message + "<|eot_id|>" +  # System Message

        "<|start_header_id|>user<|end_header_id|>\n\n" + f"Translate the English input text into Luxembourgish.\n{input_text}. Do not include any additional information or unrelated content."  +  "<|eot_id|><|start_header_id|>assistant<|end_header_id|>" # User Query
    )

# full_prompt = f"Translate the English input text into Luxembourgish.\n{input_text}. Do not include any additional information or unrelated content."

# Tokenize input text
encoded_input = tokenizer(full_prompt, return_tensors="pt", add_special_tokens=True)
model_inputs = encoded_input.to(device)
generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512 * 2,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
        temperature=1.0,
    )
decoded_output = tokenizer.batch_decode(generated_ids)[0].replace(full_prompt, "")

special_tokens = tokenizer.all_special_tokens
for token in special_tokens:
    decoded_output = decoded_output.replace(token, "")
    
print("Test Output:\n", decoded_output)


Test Output:
 D'Konzept vun dem Fraktgesaga ass net ëmdee konservative. Et ass eng d’Verwaltungsschad vun der Staats- a Regierung afunge. Zu këtten ass den europäesche Unioun, déi an de Memberstaaten heescht, fir deemno sinn d'Aachtergëtt, d'Wirtschaftspolitik, die Monetarischoft, d'Zivilsicherheet a d'Internationaledes Kämpfungsregeln a war um Fall. Déi Unioun baut no deemno mam Memberstaaten Déi an deemno sinn d’Aachtergëtt fir den Interessen vu.
